<a href="https://colab.research.google.com/github/Hirusha99/AWS/blob/main/Predictions_AWS_Sagemaker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Importing Important Libraries

In [ ]:
import sagemaker
import boto3
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.session import s3_input, Session

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [ ]:
bucket_name = 'bankappication' # unique name for bucket
my_region = boto3.session.Session().region_name # set region of the instance
print(my_region)

us-east-1


In [ ]:
s3 = boto3.resource('s3')

try:
    if my_region == 'us-east-1':
        s3.create_bucket(Bucket = bucket_name)
    print('S3 Bucket created successfully!')
except Exception as e:
    print(" S3 error :", e)

S3 Bucket created successfully!


In [ ]:
# set output path where the trained model will be save
prefix = 'xgboost-as-a-built-in-algo' # algorithm
output_path = 's3://{}/{}/output'.format(bucket_name,prefix)
print(output_path)

s3://bankappication/xgboost-as-a-built-in-algo/output


### Downloading the dataSet and storing in S3

In [ ]:
import pandas as pd
import urllib

try:

    print('Success : downloaded bank_clean.csv')
except Exception as e :
    print('Data load error :',e)

try :
    model_data = pd.read_csv('./bank_clean.csv',index_col=0) # index_col -> first column of the CSV file should be used as the index of the DataFrame.
    print('Success: Data loaded into dataframe.')
except Exception as e:
    print("Data load error:",e)

Success : downloaded bank_clean.csv
Success: Data loaded into dataframe.


In [ ]:
### train test split

import numpy as np
# value of 1 for frac means that you want to include all rows in your sample.
train_data , test_data = np.split(model_data.sample(frac = 1 , random_state = 1729),[int(0.7 * len (model_data))]) # training size should be 70%
print(train_data.shape,test_data.shape)


(28831, 61) (12357, 61)


In [ ]:
import os
pd.concat([train_data['y_yes'],train_data.drop(['y_yes','y_no'],
                                              axis = 1)],
         axis = 1).to_csv('train.csv',index = False, header = False)

boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix,'train/train.csv')).upload_file('train.csv')

s3_input_train = sagemaker.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')


In [ ]:
pd.concat([test_data['y_yes'],test_data.drop(['y_yes','y_no'],
                                              axis = 1)],
         axis = 1).to_csv('test.csv',index = False, header = False)

boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix,'test/test.csv')).upload_file('test.csv')

s3_input_test = sagemaker.TrainingInput(s3_data='s3://{}/{}/test'.format(bucket_name, prefix), content_type='csv')

### Building model Xgboost - Inbuilt Algorithm

In [ ]:
# this line automatically looks for the XGBoost image URI and builds an XGBoost container
# specify the repo_version depending on your preference

container = get_image_uri(boto3.Session().region_name,
                         'xgboost',
                         repo_version= '1.0-1')

The method get_image_uri has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [ ]:
# initialize hyperparameter
hyperparameters = {
    "max_depth":"5",
    "eta": "0.2",
    "gamma" : "4",
    "min_child_weight": "6",
    "subsample": "0.7",
    "objective":"binary:logistic",
}

In [ ]:
# construct a SageMaker estimator that calls the xgboost-container
estimator = sagemaker.estimator.Estimator(image_uri=container,
                                          hyperparameters=hyperparameters,
                                          role=sagemaker.get_execution_role(),
                                          instance_type='ml.t3.medium',
                                          instance_count=1,
                                          volume_size=5, # 5 GB
                                          output_path=output_path,
                                          use_spot_instances = True,
                                          max_run = 300,
                                          max_wait = 600
                                        )

In [ ]:
estimator.fit({'train': s3_input_train,'validation': s3_input_test})

INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2024-02-14-09-58-17-629


ClientError: An error occurred (ValidationException) when calling the CreateTrainingJob operation: 1 validation error detected: Value 'ml.t3.medium' at 'resourceConfig.instanceType' failed to satisfy constraint: Member must satisfy enum value set: [ml.m6i.xlarge, ml.trn1.32xlarge, ml.p2.xlarge, ml.m5.4xlarge, ml.m4.16xlarge, ml.m6i.12xlarge, ml.p5.48xlarge, ml.m6i.24xlarge, ml.p4d.24xlarge, ml.g5.2xlarge, ml.c5n.xlarge, ml.p3.16xlarge, ml.m5.large, ml.m6i.16xlarge, ml.p2.16xlarge, ml.g5.4xlarge, ml.c4.2xlarge, ml.c5.2xlarge, ml.c6i.32xlarge, ml.c4.4xlarge, ml.c6i.xlarge, ml.g5.8xlarge, ml.c5.4xlarge, ml.c6i.12xlarge, ml.c5n.18xlarge, ml.g4dn.xlarge, ml.c6i.24xlarge, ml.g4dn.12xlarge, ml.c4.8xlarge, ml.g4dn.2xlarge, ml.c6i.2xlarge, ml.c6i.16xlarge, ml.c5.9xlarge, ml.g4dn.4xlarge, ml.c6i.4xlarge, ml.c5.xlarge, ml.g4dn.16xlarge, ml.c4.xlarge, ml.trn1n.32xlarge, ml.g4dn.8xlarge, ml.c6i.8xlarge, ml.g5.xlarge, ml.c5n.2xlarge, ml.g5.12xlarge, ml.g5.24xlarge, ml.c5n.4xlarge, ml.trn1.2xlarge, m

### Deploy Machine Learning Model

In [ ]:
xgb_predictor = estimator.deploy(initial_instance_count = 1,instance_type= 'ml.t3.micro')

### Prediction of the test data

In [ ]:
from sagemaker.predictor import csv_serializer
test_data_array = test_data.drop(['y_no', 'y_yes'], axis=1).values #load the data into an array
xgb_predictor.content_type = 'text/csv' # set the data type for an inference (model content type)
xgb_predictor.serializer = csv_serializer # set the serializer type
predictions = xgb_predictor.predict(test_data_array).decode('utf-8') # predict!
predictions_array = np.fromstring(predictions[1:], sep=',') # and turn the prediction into an array
print(predictions_array.shape)

In [ ]:
predictions_array

In [ ]:
# confusion matrix

cm = pd.crosstab(index=test_data['y_yes'], columns=np.round(predictions_array), rownames=['Observed'], colnames=['Predicted'])
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Purchase", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Purchase", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))

### Delete Endpoints

In [ ]:
#sagemaker.Session().delete_endpoint(xgb_predictor.endpoint)
bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)
bucket_to_delete.objects.all().delete()

[{'ResponseMetadata': {'RequestId': 'DJZYRCGMQ7QHEHD0',
   'HostId': 'FHmEveIiZE/zz8IO4BOtlXQPyh8iYcinufr6D2oa9FYqzl1z282xpxr8wpxv1xdUv0fDW0gI3TVM5iqoumkk8A==',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': 'FHmEveIiZE/zz8IO4BOtlXQPyh8iYcinufr6D2oa9FYqzl1z282xpxr8wpxv1xdUv0fDW0gI3TVM5iqoumkk8A==',
    'x-amz-request-id': 'DJZYRCGMQ7QHEHD0',
    'date': 'Wed, 14 Feb 2024 11:00:22 GMT',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3',
    'connection': 'close'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'xgboost-as-a-built-in-algo/train/train.csv'},
   {'Key': 'xgboost-as-a-built-in-algo/test/test.csv'}]}]